## Install Packages

In [1]:
try:
    import pymongo
except:
    !pip install pymongo

try:
    import folium
except:
    !pip install folium

     |████████████████████████████████| 94 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 15.4 MB 9.0 MB/s eta 0:00:01     |███████████████████████▎        | 11.2 MB 3.9 MB/s eta 0:00:02


## Conncet to MongoDB

In [13]:
from pymongo import MongoClient
client = MongoClient('mongo:27017', 
                        username='root',
                         password='example')

# client = MongoClient('localhost:27017', username='root',password='example') # In case access from another instance  

## list of all databases

In [14]:
for db in client.list_databases():
    print(db)

{'name': 'admin', 'sizeOnDisk': 163840.0, 'empty': False}
{'name': 'config', 'sizeOnDisk': 12288.0, 'empty': False}
{'name': 'covid19_db2020', 'sizeOnDisk': 40960.0, 'empty': False}
{'name': 'local', 'sizeOnDisk': 73728.0, 'empty': False}
{'name': 'tamer', 'sizeOnDisk': 139264.0, 'empty': False}


In [15]:
for db in client.list_database_names():
    print(db)

admin
config
covid19_db2020
local
tamer


## Create DataBase

In [16]:
db = client['covid19_db2020']

## Create Collection 

In [17]:
cases = db['cases_col_2020']


## Read json file 

In [18]:
!ls data_json |wc -l

35502


In [19]:
!ls data_json | head -n 10

0.json
10000.json
10001.json
10002.json
10003.json
10004.json
10005.json
10006.json
10007.json
10008.json
ls: write error: Broken pipe


In [20]:
import json 
with open('data_json/1.json', 'r') as f:
    file_dict=json.loads(json.load(f))

In [21]:
file_dict

{'Date': '3/8/2020',
 'Country_Region': 'India',
 'Province_State': None,
 'Difference': 0,
 'Prep_Flow_Runtime': '3/24/2020 9:39:03 AM',
 'Latest_Date': '3/23/2020',
 'Case_Type': 'Deaths',
 'Cases': 0,
 'Lat': 21.0,
 'Long': 78.0,
 'Index': 1}

In [22]:
!ls data_json | wc -l

35502


# Push documents to collection  

In [23]:
result = cases.insert_one(file_dict)

# Push all files 

In [24]:
import glob

List=glob.glob('data_json/*.json')

In [25]:
List[0:10]

['data_json/0.json',
 'data_json/1.json',
 'data_json/10.json',
 'data_json/100.json',
 'data_json/1000.json',
 'data_json/10000.json',
 'data_json/10001.json',
 'data_json/10002.json',
 'data_json/10003.json',
 'data_json/10004.json']

In [26]:
len(List)

35502

In [27]:
from tqdm import tqdm
for file in tqdm(List[1:1001]):
    with open(file, 'r') as f:
        file_dict=json.loads(json.load(f))
        
    result = cases.insert_one(file_dict)
    

100%|██████████| 1000/1000 [00:14<00:00, 71.04it/s]


## Return all the documents in a collection in mongodb

In [29]:
cursor = cases.find({'Date': '3/8/2020'})
for document in cursor:
    print(document)

{'_id': ObjectId('607dfc608bd8aedc176aecc7'), 'Date': '3/8/2020', 'Country_Region': 'India', 'Province_State': None, 'Difference': 0, 'Prep_Flow_Runtime': '3/24/2020 9:39:03 AM', 'Latest_Date': '3/23/2020', 'Case_Type': 'Deaths', 'Cases': 0, 'Lat': 21.0, 'Long': 78.0, 'Index': 1}
{'_id': ObjectId('607dfc818bd8aedc176aecc9'), 'Date': '3/8/2020', 'Country_Region': 'India', 'Province_State': None, 'Difference': 0, 'Prep_Flow_Runtime': '3/24/2020 9:39:03 AM', 'Latest_Date': '3/23/2020', 'Case_Type': 'Deaths', 'Cases': 0, 'Lat': 21.0, 'Long': 78.0, 'Index': 1}
{'_id': ObjectId('607dfc958bd8aedc176aecca'), 'Date': '3/8/2020', 'Country_Region': 'India', 'Province_State': None, 'Difference': 0, 'Prep_Flow_Runtime': '3/24/2020 9:39:03 AM', 'Latest_Date': '3/23/2020', 'Case_Type': 'Deaths', 'Cases': 0, 'Lat': 21.0, 'Long': 78.0, 'Index': 1}
{'_id': ObjectId('607dfc958bd8aedc176aecd3'), 'Date': '3/8/2020', 'Country_Region': 'Cabo Verde', 'Province_State': None, 'Difference': 0, 'Prep_Flow_Runtime

In [33]:
cursor = cases.find({})
Locations=[]
Name=[]
for document in tqdm(cursor):
    #print(document['Lat'],document['Long'],document['Case_Type'])
    Locations.append([document['Lat'],document['Long']])
    Name.append(document['Case_Type'])

1002it [00:00, 24258.67it/s]


In [30]:
import folium
map = folium.Map(location=[38.9, -77.05], zoom_start=8)
folium.Marker([38.9, -77.05], 
                  popup='PSUT is the best'
                  ).add_to(map)
folium.Marker([38.9, -77.2], 
                  popup='PSUT is the best always'
                  ).add_to(map)
map

In [31]:
map.save('Mymap.html')

In [34]:
import folium
map = folium.Map(location=[38.9, -77.05], zoom_start=2)
for point in range(0, len(Name)):
    folium.Marker(Locations[point], 
                  popup=Name[point]
                  ).add_to(map)
map

In [35]:
import folium
map = folium.Map(location=[38.9, -77.05],tiles="Stamen Terrain", zoom_start=2)
for point in range(0, len(Name)):
    folium.Marker(Locations[point], 
                  popup=Name[point]
                  ).add_to(map)
map



In [36]:


import folium
map = folium.Map(location=[38.9, -77.05],tiles="Stamen Toner", zoom_start=2)
for point in range(0, len(Name)):
    folium.Marker(Locations[point], 
                  popup=Name[point]
                  ).add_to(map)
map


In [52]:
map.save('CovideCases.html')

In [53]:
!du -sh data_json

0	data_json
